## Carregando os datasets

In [2]:
import pandas as pd

filmes = pd.read_csv('movies.csv')
filmes.columns = ["filmeId", "título", "gêneros"]

# O índice da tabela exibida de filmes será filmeId, não um
# autoincremento 
filmes = filmes.set_index("filmeId")
filmes.head()

título  \
filmeId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                             gêneros  
filmeId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy

In [3]:
notas = pd.read_csv("ratings.csv")
notas.columns = ["usuarioId", "filmeId", "nota", "momento"]
notas.head()

usuarioId  filmeId  nota    momento
0          1        1   4.0  964982703
1          1        3   4.0  964981247
2          1        6   4.0  964982224
3          1       47   5.0  964983815
4          1       50   5.0  964982931

In [4]:
notas.describe()

usuarioId        filmeId           nota       momento
count  100836.000000  100836.000000  100836.000000  1.008360e+05
mean      326.127564   19435.295718       3.501557  1.205946e+09
std       182.618491   35530.987199       1.042529  2.162610e+08
min         1.000000       1.000000       0.500000  8.281246e+08
25%       177.000000    1199.000000       3.000000  1.019124e+09
50%       325.000000    2991.000000       3.500000  1.186087e+09
75%       477.000000    8122.000000       4.000000  1.435994e+09
max       610.000000  193609.000000       5.000000  1.537799e+09

## Primeira recomendação

In [5]:
# Como não conhecemos em nada o usuário, que tal recomendarmos primeiramente o filme mais avaliado?
# Bora contar o filme que mais pessoas deram uma nota? Recomendar um mais popular pode ser uma boa!

total_de_votos = notas["filmeId"].value_counts()
total_de_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [6]:
# Repare que o filme com id 356 foi o mais recomendado, mas, qual filme é esse exatamente?

filmes.loc[356]

título          Forrest Gump (1994)
gêneros    Comedy|Drama|Romance|War
Name: 356, dtype: object

In [7]:
# Então, listar os filmes mais avaliados pode ser uma boa ideia já que estamos começando de uma partida fria
# Se conhecessemos os usuários, poderíamos listar filmes mais específicos como os brasileiros, por exemplo
# Vamos juntar? É possível através do índice unir filmes e notas (total de votos)

filmes["total_de_votos"] = total_de_votos
filmes.head()

título  \
filmeId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                             gêneros  total_de_votos  
filmeId                                                               
1        Adventure|Animation|Children|Comedy|Fantasy           215.0  
2                         Adventure|Children|Fantasy           110.0  
3                                     Comedy|Romance            52.0  
4                               Comedy|Drama|Romance             7.0  
5                                             Comedy            49.0

### Primeira recomendação, filmes mais votados pelos usuários

In [8]:
# Ordena os filmes baseados em uma heurística de valor do total de votos
filmes.sort_values("total_de_votos", ascending = False).head(10)

título  \
filmeId                                              
356                            Forrest Gump (1994)   
318               Shawshank Redemption, The (1994)   
296                            Pulp Fiction (1994)   
593               Silence of the Lambs, The (1991)   
2571                            Matrix, The (1999)   
260      Star Wars: Episode IV - A New Hope (1977)   
480                           Jurassic Park (1993)   
110                              Braveheart (1995)   
589              Terminator 2: Judgment Day (1991)   
527                        Schindler's List (1993)   

                                  gêneros  total_de_votos  
filmeId                                                    
356              Comedy|Drama|Romance|War           329.0  
318                           Crime|Drama           317.0  
296           Comedy|Crime|Drama|Thriller           307.0  
593                 Crime|Horror|Thriller           279.0  
2571               Action|Sci-Fi|Thriller           278.0  
260               Action|Adventure|Sci-Fi           251.0  
480      Action|Adventure|Sci-Fi|Thriller           238.0  
110                      Action|Drama|War           237.0  
589                         Action|Sci-Fi           224.0  
527                             Drama|War           220.0

### Segunda recomendação, filmes mais bem avaliados pelos usuários

In [9]:
notas_medias = notas.groupby("filmeId").mean()["nota"]
filmes["notas_medias"] = notas_medias

filmes.sort_values("total_de_votos", ascending = False).head(10)

título  \
filmeId                                              
356                            Forrest Gump (1994)   
318               Shawshank Redemption, The (1994)   
296                            Pulp Fiction (1994)   
593               Silence of the Lambs, The (1991)   
2571                            Matrix, The (1999)   
260      Star Wars: Episode IV - A New Hope (1977)   
480                           Jurassic Park (1993)   
110                              Braveheart (1995)   
589              Terminator 2: Judgment Day (1991)   
527                        Schindler's List (1993)   

                                  gêneros  total_de_votos  notas_medias  
filmeId                                                                  
356              Comedy|Drama|Romance|War           329.0      4.164134  
318                           Crime|Drama           317.0      4.429022  
296           Comedy|Crime|Drama|Thriller           307.0      4.197068  
593                 Crime|Horror|Thriller           279.0      4.161290  
2571               Action|Sci-Fi|Thriller           278.0      4.192446  
260               Action|Adventure|Sci-Fi           251.0      4.231076  
480      Action|Adventure|Sci-Fi|Thriller           238.0      3.750000  
110                      Action|Drama|War           237.0      4.031646  
589                         Action|Sci-Fi           224.0      3.970982  
527                             Drama|War           220.0      4.225000

In [10]:
# Ops, com os resultados acima conseguimos perceber que os filmes mais votados não necessáriamente são 
# aqueles com as melhores notas. Apenas detectamos quais os usuários deram mais notas (que podem ser notas péssimas)
# Que tal listar os filmes pelas melhores notas?

filmes.sort_values("notas_medias", ascending = False).head(10)

título                   gêneros  \
filmeId                                                                        
88448        Paper Birds (Pájaros de papel) (2010)              Comedy|Drama   
100556                  Act of Killing, The (2012)               Documentary   
143031                             Jump In! (2007)      Comedy|Drama|Romance   
143511                                Human (2015)               Documentary   
143559                         L.A. Slasher (2015)      Comedy|Crime|Fantasy   
6201                              Lady Jane (1986)             Drama|Romance   
102217              Bill Hicks: Revelations (1993)                    Comedy   
102084                Justice League: Doom (2012)   Action|Animation|Fantasy   
6192     Open Hearts (Elsker dig for evigt) (2002)                   Romance   
145994                      Formula of Love (1984)                    Comedy   

         total_de_votos  notas_medias  
filmeId                                
88448               1.0           5.0  
100556              1.0           5.0  
143031              1.0           5.0  
143511              1.0           5.0  
143559              1.0           5.0  
6201                1.0           5.0  
102217              1.0           5.0  
102084              1.0           5.0  
6192                1.0           5.0  
145994              1.0           5.0

In [11]:
# Legal! Conseguimos os filmes nota 5, mas, se pararmos para reparar a grande parte são filmes pouquíssimo conhecidos.
# Bastante específico, de nichos. Usar apenas a nota média não parece ser algo interessante.
# Parece que tem algo estranho. Quando olhamos para o total de votos vemos que UMA única pessoa votou neles com 5. 
# Um único voto não representa a opinião da maioria e acaba tornando injusta a recomendação já que excluímos milhares 
# de outras opiniões! Precisamos filtar filmes que um determinado total de votos é maior que 10, por exemplo.

### Terceira recomendação, filmes mais bem avaliados pelos usuários e com um determinado percentual de votos

In [12]:
filmes.query("total_de_votos > = 10").sort_values("notas_medias", ascending = False).head(10)

título           gêneros  \
filmeId                                                            
1041                     Secrets & Lies (1996)             Drama   
3451       Guess Who's Coming to Dinner (1967)             Drama   
1178                     Paths of Glory (1957)         Drama|War   
1104          Streetcar Named Desire, A (1951)             Drama   
2360          Celebration, The (Festen) (1998)             Drama   
1217                                Ran (1985)         Drama|War   
318           Shawshank Redemption, The (1994)       Crime|Drama   
951                     His Girl Friday (1940)    Comedy|Romance   
1927     All Quiet on the Western Front (1930)  Action|Drama|War   
3468                       Hustler, The (1961)             Drama   

         total_de_votos  notas_medias  
filmeId                                
1041               11.0      4.590909  
3451               11.0      4.545455  
1178               12.0      4.541667  
1104               20.0      4.475000  
2360               12.0      4.458333  
1217               15.0      4.433333  
318               317.0      4.429022  
951                14.0      4.392857  
1927               10.0      4.350000  
3468               18.0      4.333333

In [13]:
# É... Melhorou um pouco mas ainda existem muitos valores nichados. Muitos filmes com 10 votos ou próximo disso. 
# Que tal aumentarmos o filtro para 50? 

filmes.query("total_de_votos >= 50").sort_values("notas_medias", ascending = False).head(10)

título  \
filmeId                                                      
318                       Shawshank Redemption, The (1994)   
858                                  Godfather, The (1972)   
2959                                     Fight Club (1999)   
1276                                 Cool Hand Luke (1967)   
750      Dr. Strangelove or: How I Learned to Stop Worr...   
904                                     Rear Window (1954)   
1221                        Godfather: Part II, The (1974)   
48516                                 Departed, The (2006)   
1213                                     Goodfellas (1990)   
912                                      Casablanca (1942)   

                             gêneros  total_de_votos  notas_medias  
filmeId                                                             
318                      Crime|Drama           317.0      4.429022  
858                      Crime|Drama           192.0      4.289062  
2959     Action|Crime|Drama|Thriller           218.0      4.272936  
1276                           Drama            57.0      4.271930  
750                       Comedy|War            97.0      4.268041  
904                 Mystery|Thriller            84.0      4.261905  
1221                     Crime|Drama           129.0      4.259690  
48516           Crime|Drama|Thriller           107.0      4.252336  
1213                     Crime|Drama           126.0      4.250000  
912                    Drama|Romance           100.0      4.240000

In [14]:
# Já melhorou bastante a filtragem, mas, continua com valores próximos de 50 votos. O que ainda acusa nicho nos 
# resultados. Recomendando um conteúdo muito bem avaliado por um nicho pequeno, as chances de erro com um usuário que não 
# conhecemos absolutamente nada sobre podem ser imensas. Que tal consideraros as melhores notas nos mais bem avaliados?
# Assim combinamos duas heurísticas para a recomendação!

In [20]:
filmes.query("total_de_votos >= 50").sort_values(["notas_medias", "total_de_votos"], ascending = [False, False]).head(10)

título  \
filmeId                                                      
318                       Shawshank Redemption, The (1994)   
858                                  Godfather, The (1972)   
2959                                     Fight Club (1999)   
1276                                 Cool Hand Luke (1967)   
750      Dr. Strangelove or: How I Learned to Stop Worr...   
904                                     Rear Window (1954)   
1221                        Godfather: Part II, The (1974)   
48516                                 Departed, The (2006)   
1213                                     Goodfellas (1990)   
912                                      Casablanca (1942)   

                             gêneros  total_de_votos  notas_medias  
filmeId                                                             
318                      Crime|Drama           317.0      4.429022  
858                      Crime|Drama           192.0      4.289062  
2959     Action|Crime|Drama|Thriller           218.0      4.272936  
1276                           Drama            57.0      4.271930  
750                       Comedy|War            97.0      4.268041  
904                 Mystery|Thriller            84.0      4.261905  
1221                     Crime|Drama           129.0      4.259690  
48516           Crime|Drama|Thriller           107.0      4.252336  
1213                     Crime|Drama           126.0      4.250000  
912                    Drama|Romance           100.0      4.240000

In [16]:
# O melhor que conseguimos chegar sem conhecer nada dos usuários, apenas considerando o número de avaliações
# e as notas dadas pelos usuários é isso. Conseguimos um top 10 filmes mais bem "avaliados" pela maioria. 
# Se subimos o filtro de avaliações, os filmes mais antigos (que mais pessoas tiveram a oportunidade de assistir) 
# irão sempre prevalecer sobre os recém lançados mesmo que tenham um número razoável de avaliações e notas altas.
# Repare que ainda sim existem valores baixos, como 57 votos ou 100 votos.

# Resumo do problema, encontrar um ponto de equilíbrio sem conhecer nada do usuário:
# Se forçarmos um número de votos maior, automaticamente podemos sugerir filmes pior avaliados.
# E, se forçarmos um número de notas mais altas, podemos recomendar filmes que pouquíssimas pessoas avaliaram e que
# não necessariamente expressa a opinião da maioria. E aqui, chegamos ao grande dilema das recomendações "as cegas"